In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspark-sample-dataset/sample_data.csv


In [2]:
!pip install findspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 28 kB/s              
  Preparing metadata (setup.py) ... - \ done
     |████████████████████████████████| 198 kB 62.9 MB/s            
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=978be37bbc450be337ba84a8531410ee67ee1f404ec316ae21d045568413521c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.4
    Uninstalling py4j-0.10.9.4:
      Successfully uninstalled py4j-0.10.9.4


In [3]:
import findspark
findspark.init()
import pyspark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/06 08:29:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import pandas_udf

In [6]:
df = spark.read.csv('../input/pyspark-sample-dataset', inferSchema=True, header=True)
df.show()

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
|      4| 27|       9.0|     0|   Oppo|
|      5| 37|      23.0|     5|   Vivo|
|      5| 37|      23.0|     5|Samsung|
|      3| 22|       2.5|     0|  Apple|
|      3| 27|       6.0|     0|     MI|
|      2| 27|       6.0|     2|   Oppo|
|      5| 27|       6.0|     2|Samsung|
|      3| 37|      16.5|     5|  Apple|
|      5| 27|       6.0|     0|     MI|
|      4| 22|       6.0|     1|   Oppo|
|      4| 37|       9.0|     2|Samsung|
|      4| 27|       6.0|     1|  Apple|
|      1| 37|      23.0|     5|     MI|
|      2| 42|      23.0|     2|   Oppo|
|      4| 37|       6.0|     0|   Vivo|
+-------+---+----------+------+-------+
only showing top 20 rows



In [7]:
#check the columns
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [8]:
# number of columns
len(df.columns)

5

In [9]:
#no of rows
df.count()

33

In [10]:
#since in pyspark, there is no direct method to get the shape, we use the combination of the two,
print('The shape of the dataframe is:({},{})'.format((df.count()),(len(df.columns))))

The shape of the dataframe is:(33,5)


In [11]:
#the datatypes of the columns are as follows:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [12]:
#to display the first 5 rows
df.show(5)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
+-------+---+----------+------+-------+
only showing top 5 rows



In [13]:
#lets check only the 2 columns age & mobile, see only 5 rows
df.select('age', 'mobile').show(5)

+---+-------+
|age| mobile|
+---+-------+
| 32|   Vivo|
| 27|  Apple|
| 22|Samsung|
| 37|  Apple|
| 27|     MI|
+---+-------+
only showing top 5 rows



In [14]:
#summary stats
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  null|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  null|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



In [15]:
#add a new columns, which is the current column plus 10
df = df.withColumn('age_after_10_years', (df['age']+10))
df.show(10, False)

+-------+---+----------+------+-------+------------------+
|ratings|age|experience|family|mobile |age_after_10_years|
+-------+---+----------+------+-------+------------------+
|3      |32 |9.0       |3     |Vivo   |42                |
|3      |27 |13.0      |3     |Apple  |37                |
|4      |22 |2.5       |0     |Samsung|32                |
|4      |37 |16.5      |4     |Apple  |47                |
|5      |27 |9.0       |1     |MI     |37                |
|4      |27 |9.0       |0     |Oppo   |37                |
|5      |37 |23.0      |5     |Vivo   |47                |
|5      |37 |23.0      |5     |Samsung|47                |
|3      |22 |2.5       |0     |Apple  |32                |
|3      |27 |6.0       |0     |MI     |37                |
+-------+---+----------+------+-------+------------------+
only showing top 10 rows



In [16]:
#change the columns age dtype to double

df = df.withColumn('age_Double', df['age'].cast(DoubleType()))
df.show(3, False)

+-------+---+----------+------+-------+------------------+----------+
|ratings|age|experience|family|mobile |age_after_10_years|age_Double|
+-------+---+----------+------+-------+------------------+----------+
|3      |32 |9.0       |3     |Vivo   |42                |32.0      |
|3      |27 |13.0      |3     |Apple  |37                |27.0      |
|4      |22 |2.5       |0     |Samsung|32                |22.0      |
+-------+---+----------+------+-------+------------------+----------+
only showing top 3 rows



In [17]:
#Filter, there are 2 ways for filter
#lets filter only Vivo
df.filter(df['mobile']=='Vivo').show(5, False)

+-------+---+----------+------+------+------------------+----------+
|ratings|age|experience|family|mobile|age_after_10_years|age_Double|
+-------+---+----------+------+------+------------------+----------+
|3      |32 |9.0       |3     |Vivo  |42                |32.0      |
|5      |37 |23.0      |5     |Vivo  |47                |37.0      |
|4      |37 |6.0       |0     |Vivo  |47                |37.0      |
|5      |37 |13.0      |1     |Vivo  |47                |37.0      |
|4      |37 |6.0       |0     |Vivo  |47                |37.0      |
+-------+---+----------+------+------+------------------+----------+



In [18]:
#adding 2 filter conditions--method#1
df.filter(df['mobile']=='Vivo').filter(df['experience']>10).show()

+-------+---+----------+------+------+------------------+----------+
|ratings|age|experience|family|mobile|age_after_10_years|age_Double|
+-------+---+----------+------+------+------------------+----------+
|      5| 37|      23.0|     5|  Vivo|                47|      37.0|
|      5| 37|      13.0|     1|  Vivo|                47|      37.0|
+-------+---+----------+------+------+------------------+----------+



In [19]:
#method #2, 2 conditions in one row.
df.filter((df['mobile']=='Vivo') & (df['experience'] >10)).show()

+-------+---+----------+------+------+------------------+----------+
|ratings|age|experience|family|mobile|age_after_10_years|age_Double|
+-------+---+----------+------+------+------------------+----------+
|      5| 37|      23.0|     5|  Vivo|                47|      37.0|
|      5| 37|      13.0|     1|  Vivo|                47|      37.0|
+-------+---+----------+------+------+------------------+----------+



In [20]:
#checking distinct values in column
df.select('mobile').distinct().show()

+-------+
| mobile|
+-------+
|     MI|
|   Oppo|
|Samsung|
|   Vivo|
|  Apple|
+-------+



In [21]:
#grouping the data
df.groupby('mobile').count().show(5, False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Oppo   |7    |
|Samsung|6    |
|Vivo   |5    |
|Apple  |7    |
+-------+-----+



In [22]:
#groupby ascending
df.groupby('mobile').count().orderBy('count', ascending=False).show(5, False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Apple  |7    |
|Oppo   |7    |
|Samsung|6    |
|Vivo   |5    |
+-------+-----+



In [23]:
#checking the mean
df.groupby('mobile').mean().show(5, False)

+-------+------------------+------------------+------------------+------------------+-----------------------+------------------+
|mobile |avg(ratings)      |avg(age)          |avg(experience)   |avg(family)       |avg(age_after_10_years)|avg(age_Double)   |
+-------+------------------+------------------+------------------+------------------+-----------------------+------------------+
|MI     |3.5               |30.125            |10.1875           |1.375             |40.125                 |30.125            |
|Oppo   |2.857142857142857 |28.428571428571427|10.357142857142858|1.4285714285714286|38.42857142857143      |28.428571428571427|
|Samsung|4.166666666666667 |28.666666666666668|8.666666666666666 |1.8333333333333333|38.666666666666664     |28.666666666666668|
|Vivo   |4.2               |36.0              |11.4              |1.8               |46.0                   |36.0              |
|Apple  |3.4285714285714284|30.571428571428573|11.0              |2.7142857142857144|40.571428571

In [24]:
#chekcing the sum
df.groupby('mobile').sum().show(5, False)

+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|mobile |sum(ratings)|sum(age)|sum(experience)|sum(family)|sum(age_after_10_years)|sum(age_Double)|
+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|MI     |28          |241     |81.5           |11         |321                    |241.0          |
|Oppo   |20          |199     |72.5           |10         |269                    |199.0          |
|Samsung|25          |172     |52.0           |11         |232                    |172.0          |
|Vivo   |21          |180     |57.0           |9          |230                    |180.0          |
|Apple  |24          |214     |77.0           |19         |284                    |214.0          |
+-------+------------+--------+---------------+-----------+-----------------------+---------------+



In [25]:
#chekcing min
df.groupBy('mobile').min().show(5, False)

+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|mobile |min(ratings)|min(age)|min(experience)|min(family)|min(age_after_10_years)|min(age_Double)|
+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|MI     |1           |27      |2.5            |0          |37                     |27.0           |
|Oppo   |2           |22      |6.0            |0          |32                     |22.0           |
|Samsung|2           |22      |2.5            |0          |32                     |22.0           |
|Vivo   |3           |32      |6.0            |0          |42                     |32.0           |
|Apple  |3           |22      |2.5            |0          |32                     |22.0           |
+-------+------------+--------+---------------+-----------+-----------------------+---------------+



In [26]:
#chekcing max
df.groupby('mobile').max().show(5, False)

+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|mobile |max(ratings)|max(age)|max(experience)|max(family)|max(age_after_10_years)|max(age_Double)|
+-------+------------+--------+---------------+-----------+-----------------------+---------------+
|MI     |5           |42      |23.0           |5          |52                     |42.0           |
|Oppo   |4           |42      |23.0           |2          |52                     |42.0           |
|Samsung|5           |37      |23.0           |5          |47                     |37.0           |
|Vivo   |5           |37      |23.0           |5          |47                     |37.0           |
|Apple  |4           |37      |16.5           |5          |47                     |37.0           |
+-------+------------+--------+---------------+-----------+-----------------------+---------------+



In [27]:
#we can also use aggregation to check  
df.groupby('mobile').agg({'experience':'sum'}).show(5, False)

+-------+---------------+
|mobile |sum(experience)|
+-------+---------------+
|MI     |81.5           |
|Oppo   |72.5           |
|Samsung|52.0           |
|Vivo   |57.0           |
|Apple  |77.0           |
+-------+---------------+



In [28]:
#udf
#this one is done in 3 steps
# Step #1 write a trational python function
def price_rangefunction(brand):
    if brand in ['Samsung', 'Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

# Step #2 , create a UDF
brand_udf = udf(price_rangefunction, StringType())

#step #3, apply the udf to mobile column
df.withColumn('price_range', brand_udf(df['mobile'])).show(10, False)


+-------+---+----------+------+-------+------------------+----------+-----------+
|ratings|age|experience|family|mobile |age_after_10_years|age_Double|price_range|
+-------+---+----------+------+-------+------------------+----------+-----------+
|3      |32 |9.0       |3     |Vivo   |42                |32.0      |Low Price  |
|3      |27 |13.0      |3     |Apple  |37                |27.0      |High Price |
|4      |22 |2.5       |0     |Samsung|32                |22.0      |High Price |
|4      |37 |16.5      |4     |Apple  |47                |37.0      |High Price |
|5      |27 |9.0       |1     |MI     |37                |27.0      |Mid Price  |
|4      |27 |9.0       |0     |Oppo   |37                |27.0      |Low Price  |
|5      |37 |23.0      |5     |Vivo   |47                |37.0      |Low Price  |
|5      |37 |23.0      |5     |Samsung|47                |37.0      |High Price |
|3      |22 |2.5       |0     |Apple  |32                |22.0      |High Price |
|3      |27 |6.0

In [29]:
#using lamda function
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())
df.withColumn("age_group", age_udf(df.age)).show(10, False)


+-------+---+----------+------+-------+------------------+----------+---------+
|ratings|age|experience|family|mobile |age_after_10_years|age_Double|age_group|
+-------+---+----------+------+-------+------------------+----------+---------+
|3      |32 |9.0       |3     |Vivo   |42                |32.0      |senior   |
|3      |27 |13.0      |3     |Apple  |37                |27.0      |young    |
|4      |22 |2.5       |0     |Samsung|32                |22.0      |young    |
|4      |37 |16.5      |4     |Apple  |47                |37.0      |senior   |
|5      |27 |9.0       |1     |MI     |37                |27.0      |young    |
|4      |27 |9.0       |0     |Oppo   |37                |27.0      |young    |
|5      |37 |23.0      |5     |Vivo   |47                |37.0      |senior   |
|5      |37 |23.0      |5     |Samsung|47                |37.0      |senior   |
|3      |22 |2.5       |0     |Apple  |32                |22.0      |young    |
|3      |27 |6.0       |0     |MI     |3

In [30]:
#using Pnadas udf
def remainaing_yrs(age):
    return (100-age)
    
    
lenght_udf = pandas_udf(remainaing_yrs, IntegerType())

In [31]:
df.withColumn("yrs_left", lenght_udf(df['age'])).show(10, False)


+-------+---+----------+------+-------+------------------+----------+--------+
|ratings|age|experience|family|mobile |age_after_10_years|age_Double|yrs_left|
+-------+---+----------+------+-------+------------------+----------+--------+
|3      |32 |9.0       |3     |Vivo   |42                |32.0      |68      |
|3      |27 |13.0      |3     |Apple  |37                |27.0      |73      |
|4      |22 |2.5       |0     |Samsung|32                |22.0      |78      |
|4      |37 |16.5      |4     |Apple  |47                |37.0      |63      |
|5      |27 |9.0       |1     |MI     |37                |27.0      |73      |
|4      |27 |9.0       |0     |Oppo   |37                |27.0      |73      |
|5      |37 |23.0      |5     |Vivo   |47                |37.0      |63      |
|5      |37 |23.0      |5     |Samsung|47                |37.0      |63      |
|3      |22 |2.5       |0     |Apple  |32                |22.0      |78      |
|3      |27 |6.0       |0     |MI     |37           

In [32]:
#pandas udf using multiple columns
def prod(rating,exp):
    x=rating*exp
    return x

prod_udf = pandas_udf(prod, DoubleType())

In [33]:
df.withColumn("product", prod_udf(df['ratings'], df['experience'])).show(10, False)

+-------+---+----------+------+-------+------------------+----------+-------+
|ratings|age|experience|family|mobile |age_after_10_years|age_Double|product|
+-------+---+----------+------+-------+------------------+----------+-------+
|3      |32 |9.0       |3     |Vivo   |42                |32.0      |27.0   |
|3      |27 |13.0      |3     |Apple  |37                |27.0      |39.0   |
|4      |22 |2.5       |0     |Samsung|32                |22.0      |10.0   |
|4      |37 |16.5      |4     |Apple  |47                |37.0      |66.0   |
|5      |27 |9.0       |1     |MI     |37                |27.0      |45.0   |
|4      |27 |9.0       |0     |Oppo   |37                |27.0      |36.0   |
|5      |37 |23.0      |5     |Vivo   |47                |37.0      |115.0  |
|5      |37 |23.0      |5     |Samsung|47                |37.0      |115.0  |
|3      |22 |2.5       |0     |Apple  |32                |22.0      |7.5    |
|3      |27 |6.0       |0     |MI     |37                |27.0  

In [34]:
#delete the columns
df_new = df.drop('mobile')
df_new.show(10, False)

+-------+---+----------+------+------------------+----------+
|ratings|age|experience|family|age_after_10_years|age_Double|
+-------+---+----------+------+------------------+----------+
|3      |32 |9.0       |3     |42                |32.0      |
|3      |27 |13.0      |3     |37                |27.0      |
|4      |22 |2.5       |0     |32                |22.0      |
|4      |37 |16.5      |4     |47                |37.0      |
|5      |27 |9.0       |1     |37                |27.0      |
|4      |27 |9.0       |0     |37                |27.0      |
|5      |37 |23.0      |5     |47                |37.0      |
|5      |37 |23.0      |5     |47                |37.0      |
|3      |22 |2.5       |0     |32                |22.0      |
|3      |27 |6.0       |0     |37                |27.0      |
+-------+---+----------+------+------------------+----------+
only showing top 10 rows

